# Custom Vision に画像を追加する

Custom Vision SDK の パッケージをインポートします。

In [1]:
!pip install azure-cognitiveservices-vision-customvision

スクリプトの実行に必要なパッケージをインポートします。

In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch
from msrest.authentication import ApiKeyCredentials 
import numpy as np

自分のリソースの Azure エンドポイントおよびキー用の変数を作成します

In [2]:
# Replace with valid values
ENDPOINT = 'https://cvtakahashihandsonjpeast001.cognitiveservices.azure.com/'
training_key = 'f138fa8bda544d94897cc4a794de4141'
project_name = 'takahashi-project-001'

## クライアントを認証する

エンドポイントとキーを使用してトレーニングのクライアントをインスタンス化します。 

ドキュメント
[CustomVisionTrainingClient](https://docs.microsoft.com/ja-JP/python/api/azure-cognitiveservices-vision-customvision/azure.cognitiveservices.vision.customvision.training.customvisiontrainingclient?view=azure-python)

In [3]:
credentials = ApiKeyCredentials(in_headers={'Training-key': training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [4]:
def printMethod(obj):
    print(type(obj))
    for x in dir(obj):
        print(x, ':', type(eval('obj.'+x)))

In [5]:
# プロジェクト一覧を取得
projects = trainer.get_projects()

# 今回のプロジェクト
target_project = None

for pj in projects:
    if pj.name == project_name:
        target_project = pj

In [6]:
import os

# 鳥のタグの一覧を取得します。 タグは、ディレクトリのフォルダー名に基づいて作成します。
tags_name = [name for name in os.listdir('datasets')]
print(tags_name)

['American Crow', 'American Goldfinch (Female)', 'American Goldfinch (Male)', 'American Robin (Adult)', 'American Robin (Juvenile)', 'Blue Jay', 'Common Grackle', 'House Sparrow (Female)', 'House Sparrow (Male)', 'House Wren', 'Mourning Dove', 'Northern Cardinal (Adult Male)', 'Northern Cardinal (Female)', 'Red-tailed Hawk (Dark morph)', 'Red-tailed Hawk (Light morph immature)', 'Tufted Titmouse']


In [7]:
# Custom Vision プロジェクトにタグの作成もしくは取得が行われます。
def find_or_create_tag(project, tag_name):
    # 存在チェック
    for existing_tag in trainer.get_tags(project.id):
        if existing_tag.name == tag_name:
            return existing_tag.id

    # 新規作成
    result = trainer.create_tag(project.id, tag_name)
    print(f'{tag_name} を作成しました : {result}')
    return result.id

In [8]:
# タグ名とタグ ID を使用して画像のリストが作成されます。
def create_image_list(tag_name, tag_id):
    base_image_url = f"./datasets/{tag_name}/"
    photo_name_list = os.listdir(base_image_url)
    image_list = []
    for file_name in photo_name_list:
        with open(base_image_url+file_name, "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=base_image_url+file_name, contents=image_contents.read(), tag_ids=[tag_id]))
    return image_list

ドキュメント [create_images_from_files](https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-vision-customvision/azure.cognitiveservices.vision.customvision.training.operations.customvisiontrainingclientoperationsmixin?view=azure-python#azure-cognitiveservices-vision-customvision-training-operations-customvisiontrainingclientoperationsmixin-create-images-from-files)

In [13]:
def uploadImageList(project, images_batch):
    upload_result = trainer.create_images_from_files(project.id, batch=images_batch)
    if not upload_result.is_batch_successful:
        print("画像のアップロードに失敗しました.")
        for image in upload_result.images:
            print("ステータス: ", image.status)
        exit(-1)

In [14]:
# 画像を登録
for tag_name in tags_name:
    # タグを取得
    tag_id = find_or_create_tag(target_project, tag_name)
    # 送信する画像をリスト化
    image_list = create_image_list(tag_name, tag_id)
    print(f"{tag_name} の image_listを作成しました 合計: " + str(len(image_list)))

    # 64枚ずつ送信する(APIの上限)
    for i in range(0, len(image_list), 64):
        images_batch = ImageFileCreateBatch(images=image_list[i:i + 64])
        print(f'{tag_name} の {len(images_batch.images)}枚の画像を送信開始')
        uploadImageList(target_project, images_batch)
        print(f'{tag_name} の 送信完了')


American Crow の image_listを作成しました 合計: 117
American Crow の 64枚の画像を送信開始
American Crow の 送信完了
American Crow の 53枚の画像を送信開始
American Crow の 送信完了
American Goldfinch (Female) を作成しました : {'additional_properties': {}, 'id': '9ad936e5-b327-43ed-97a3-a600628b8b42', 'name': 'American Goldfinch (Female)', 'description': None, 'type': 'Regular', 'image_count': 0}
American Goldfinch (Female) の image_listを作成しました 合計: 120
American Goldfinch (Female) の 64枚の画像を送信開始
American Goldfinch (Female) の 送信完了
American Goldfinch (Female) の 56枚の画像を送信開始
American Goldfinch (Female) の 送信完了
American Goldfinch (Male) を作成しました : {'additional_properties': {}, 'id': 'ced97240-02e6-40c2-97a6-7504e4675c30', 'name': 'American Goldfinch (Male)', 'description': None, 'type': 'Regular', 'image_count': 0}
American Goldfinch (Male) の image_listを作成しました 合計: 120
American Goldfinch (Male) の 64枚の画像を送信開始
American Goldfinch (Male) の 送信完了
American Goldfinch (Male) の 56枚の画像を送信開始
American Goldfinch (Male) の 送信完了
American Robin (Adult) を作成しました :

## トレーニング

In [16]:
import time

print ("Training...")
iteration = trainer.train_project(target_project.id, training_type='Regular')

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(target_project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


## 操作に失敗した場合

In [11]:
# タグの全リセット
for existing_tag in trainer.get_tags(target_project.id):
    trainer.delete_tag(target_project.id, existing_tag.id)

In [ ]:
# 画像の全削除
trainer.delete_images(target_project.id)